## Bu dosya hakkında
Potansiyel bir ev alıcısının bir ev satın almadan önce düşündüğü şeyler nelerdir? Mülkün konumu, büyüklüğü, ofislerin, okulların, parkların, restoranların, hastanelerin veya basmakalıp beyaz çitlerin yakınlığı? Peki ya en önemli faktör - fiyat?

Şimdi, şeytanlaştırmanın devam eden etkisi, Emlak (Düzenleme ve Geliştirme) Yasasının (RERA) uygulanması ve şehirdeki emlak müteahhitlerine güven eksikliği nedeniyle, 2017'de Hindistan genelinde satılan konut birimleri yüzde 7 düştü. Aslında, emlak danışmanlığı Knight Frank tarafından yayınlanan bir araştırmaya göre, Bengaluru'daki emlak fiyatları 2017'nin ikinci yarısında neredeyse yüzde 5 düştü.
Örneğin, potansiyel bir ev sahibi için, 42-52 bin ₹ aralığında 9.000'den fazla apartman projesi ve satılık daire mevcuttur, ardından 52-62 bin ₹ bütçe segmentinde yer alan 7.100'den fazla daire, mülk tarafından yapılan bir rapora göre web sitesi Makaan. Araştırmaya göre, 15-25 bin ₹ bütçe segmentinde 5.000'den fazla proje var, bunu 34-43 bin ₹ bütçe kategorisinde olanlar izliyor.

Özellikle Bengaluru gibi bir şehirde ev satın almak zor bir seçimdir. Ana faktörler genellikle tüm metropoller için aynı olsa da, Hindistan'ın Silikon Vadisi için dikkate alınması gereken başka faktörler de var. Bin yıllık kalabalığı, canlı kültürü, harika iklimi ve çok sayıda iş fırsatı sayesinde, Bengaluru'da bir evin fiyatını belirlemek zordur.

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

In [4]:
import os
print("Mevcut konum: ", os.getcwd())
os.chdir('C:\\Users\\sahik\\Documents\\GitHub\\Machine_Learning_Data_Science_Projects\\data')
print("Güncel konum: ", os.getcwd())

Mevcut konum:  C:\Users\sahik\Documents\GitHub\Machine_Learning_Data_Science_Projects
Güncel konum:  C:\Users\sahik\Documents\GitHub\Machine_Learning_Data_Science_Projects\data


In [5]:
df1 = pd.read_csv("Bengaluru_House_Data.csv")
df1.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [6]:
df1.shape

(13320, 9)

In [8]:
df1.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

In [9]:
df2 = df1.drop(['area_type','society','balcony','availability'], axis='columns')
df2.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [10]:
df2.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

<div class="alert alert-block alert-info">
    Burada location da 1, size da 16 ve bath de 73 boş veri olduğunu görüyoruz. Veri setimizin toplam boyutu 13320 olduğundan dolayı bu veriler gözden çıkartılabilir. Çünkü eksik verilerin sayısı toplam veriye göre oldukça az. Bundan dolayı bir sonraki adımda bu verileri siliyoruz.
</div>

In [11]:
df3 = df2.dropna()
df3.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [13]:
df3.shape

(13246, 5)

<div class="alert alert-block alert-info">
İlk bakışta <strong>size</strong> sütununa baktığımızda <strong>BHK, Bedroom</strong> gibi farklı ifadeler olduğunu görüyoruz. Sütundaki farklı ifadelerin hepsini gözlemleyelim.
</div>

In [14]:
df3['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

<div class="alert alert-block alert-info">
Bu ifadelerde bizim için önemli kısım olan rakamları almamız gerekiyor. Bunun için boşluğa göre ifadeleri parçalayım ilk parçayı alalım ve yeni bir sütunda dairelerdeki yatak odası sayılarını gözlemleyelim.
</div>

In [21]:
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
df3.head()

C:\Users\sahik\AppData\Local\Temp\ipykernel_12112\3620022870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))


,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [22]:
df3['bhk'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

<div class="alert alert-block alert-info">
Yukarıda evlerin yatak odası sayılarını görüyoruz. 43 yatak odalı evler de varmış. Bu oldukça yüksek bir değer. 20 den fazla yatak odası olan evlere bakalım.
</div>

In [24]:
df3[df3.bhk > 20]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


<div class="alert alert-block alert-info">
43 yatak odası olan evin metrekaresinin 2400 olduğunu görüyoruz. Burada bir hata var gibi görünüyor.
</div>

<div class="alert alert-block alert-success">
<b>total_sqft</b> sütununu inceleyelim.
</div>

In [25]:
df3.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

<div class="alert alert-block alert-info">
Çıktıyı incelediğimizde '1133 - 1384' gibi aralıklı değerlerin olduğunu görüyoruz. Bu değerleri teklememiz gerekiyor. Bunun için yapmamız gereken bu iki sayının ortalamasını almak olacaktır. Bunu yapmadan önce bu sütunun tekili değerlerini floata çevirmemiz gerekiyor.
</div>

In [26]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

<div class="alert alert-block alert-warning">  " <b>~</b> "  ifadesi ile kayan sayı olmayan değerleri elde ediyoruz.</div>

In [31]:
df3[~df3['total_sqft'].apply(is_float)].head(10)

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9
661,Yelahanka,2 BHK,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,4 Bedroom,3090 - 5002,4.0,445.000,4


In [37]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1])) / 2
    try:
        return float(x)
    except:
        return None

In [38]:
convert_sqft_to_num('2166')

2166.0

In [39]:
convert_sqft_to_num('2100 - 2850')

2475.0

In [40]:
convert_sqft_to_num('34.46Sq. Meter')  # None

In [42]:
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)
df4.head(3)

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3


In [43]:
df4.loc[30]

location      Yelahanka
size              4 BHK
total_sqft       2475.0
bath                4.0
price             186.0
bhk                   4
Name: 30, dtype: object

<div class="alert alert-block alert-info">
30. satırdaki total_sqft değeri 2100 - 2850 aralığında iken artık 2475.0 değerine sahiptir.
</div>

Emlak piyasasında metre kare başına düşen fiyatı hesaplayalım.

In [48]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/ df5['total_sqft']
df5.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


<div class="alert alert-block alert-success">
    <b>location</b> sütununu inceleyelim.
</div>

In [50]:
len(df5.location.unique())

1304

<div class="alert alert-block alert-info">
1304 tane konum olduğunu görüyoruz. Bu değer metin verilerini incelemek için oldukça fazladır. Bunu sıcak kodlama kullanarak sütunlara dönüştürdüğümüzde veri setimizde yaklaşık 1300 sütun olacak. Bu kadar kolon sayısı çok fazla. Bu durum yüksek boyutlu bir problem olarak karşımıza çıkar. Konum sayısını azaltmak için burada boyut azaltma tekniğini uygulamamız gerekiyor.
</div>

In [75]:
df5.location = df5.location.apply(lambda x: x.strip())

location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

location
Whitefield               535
Sarjapur  Road           392
Electronic City          304
Kanakpura Road           266
Thanisandra              236
                        ... 
1 Giri Nagar               1
Kanakapura Road,           1
Kanakapura main  Road      1
Karnataka Shabarimala      1
whitefiled                 1
Name: location, Length: 1293, dtype: int64

In [78]:
len(location_stats[location_stats <= 10])

1052

In [79]:
location_stats_less_than_10 = location_stats[location_stats <= 10]
location_stats_less_than_10

location
Basapura                 10
1st Block Koramangala    10
Gunjur Palya             10
Kalkere                  10
Sector 1 HSR Layout      10
                         ..
1 Giri Nagar              1
Kanakapura Road,          1
Kanakapura main  Road     1
Karnataka Shabarimala     1
whitefiled                1
Name: location, Length: 1052, dtype: int64

In [80]:
len(df5.location.unique())

1293

In [81]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

242

In [83]:
df5.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000
